In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import joblib
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow_addons.optimizers import AdamW
# from tensorflow.keras.experimental import CosineDecayRestarts

In [5]:
from tensorflow.keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
# from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [6]:
# testing Dot layer for dot product
x1 = Embedding(100, output_dim=50)(np.arange(100).reshape(5, 20))
x2 = Embedding(100, output_dim=50)(np.arange(50).reshape(5, 10))
dotted = Dot(axes=2)([x1, x2])
print("(BS, n_users, embedding_size)")
print(x1.shape)
print("(BS, n_animes, embedding_size)")
print(x2.shape)
print("Dot product shape:")
print(dotted.shape)

2024-04-18 17:29:29.688694: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-18 17:29:29.688746: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-18 17:29:29.688760: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-18 17:29:29.689139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-18 17:29:29.689698: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


(BS, n_users, embedding_size)
(5, 20, 50)
(BS, n_animes, embedding_size)
(5, 10, 50)
Dot product shape:
(5, 20, 10)


In [7]:
%%time
import joblib

ENCODER_PATH = 'encoder_dicts.joblib'
encoders_dict = joblib.load(ENCODER_PATH)
print(encoders_dict.keys())

anime_id_to_idx = encoders_dict['anime_id_to_idx']
anime_idx_to_id = encoders_dict['anime_idx_to_id']
user_id_to_idx = encoders_dict['user_id_to_idx']
user_idx_to_id = encoders_dict['user_idx_to_id']

n_users, n_animes = len(user_id_to_idx), len(anime_id_to_idx)
n_users, n_animes

dict_keys(['user_id_to_idx', 'user_idx_to_id', 'anime_id_to_idx', 'anime_idx_to_id'])
CPU times: user 485 ms, sys: 31.4 ms, total: 516 ms
Wall time: 519 ms


(145311, 17562)

In [8]:
# recommended embedding sizes by fastai: (600, 363)
# reduced a lot here to make it simpler
USER_EMB_SIZE = 128
ANIME_EMB_SIZE = 128
EMBEDDING_SIZES = (USER_EMB_SIZE, ANIME_EMB_SIZE)

In [12]:
# Accelerator check
import tensorflow as tf

TPU_INIT = False

# if TPU_INIT:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
#     tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     !nvidia-smi
if TPU_INIT:
    try:
        GPU = False
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    except ValueError:
        TPU_INIT = False
        GPU = True
        print("Not connected to a TPU runtime. Using CPU/GPU strategy")
        tpu_strategy = tf.distribute.MirroredStrategy()
        !nvidia-smi
    
print(tf.__version__)

2.15.0


In [13]:
MIN_LR = 1e-5  # minimum learning rate
MAX_LR = 5e-4  # maximum learning rate
BS = 10000  # batch_size
EPOCHS = 10

if TPU_INIT:
    MAX_LR *= tpu_strategy.num_replicas_in_sync
    BS *= tpu_strategy.num_replicas_in_sync
    print("MAX_LR with TPU:", MAX_LR)
    print("batch_size with TPU:", BS)

In [14]:
FUNCTIONAL_MODEL = False

In [15]:
K = keras.backend

class RecommenderModel(Model):
    # Based on https://keras.io/examples/structured_data/collaborative_filtering_movielens/
    def __init__(self, n_users, n_animes, embedding_sizes, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.n_users = n_users
        self.n_animes = n_animes
        self.embedding_sizes = embedding_sizes
        self.user_embedding = Embedding(
            name='user_embedding',
            input_dim=n_users,
            output_dim=embedding_sizes[0],
#             embeddings_initializer='he_normal',
#             embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.anime_embedding = Embedding(
            name='anime_embedding',
            input_dim=n_animes,
            output_dim=embedding_sizes[1],
#             embeddings_initializer='he_normal',
#             embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.dot_layer = Dot(name='dot_product', normalize=True, axes=1)
        self.layers_ = [
            Flatten(),
        
#             bias is not needed when using BatchNorm
            Dense(128, activation=activation, use_bias=False, kernel_initializer='he_normal'),
            BatchNormalization(),
            Dense(64, activation=activation, use_bias=False, kernel_initializer='he_normal'),
            BatchNormalization(),
            Dense(1, use_bias=False, kernel_initializer='he_normal'),
            BatchNormalization(),
            Activation('sigmoid'),
        ]

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        anime_vector = self.anime_embedding(inputs[:, 1])
#         x = tf.concat([user_vector, anime_vector], axis=1)
        x = self.dot_layer([user_vector, anime_vector])
#         x = tf.tensordot(user_vector, anime_vector, axis=2)
        for layer in self.layers_:
            x = layer(x)
#         x = dot_user_anime + user_bias + anime_bias
        return x

def get_model():
    print("[INFO] Using Subclassing API Model")

    # Resets all state generated by Keras to clear all models
    K.clear_session()
    
    if TPU_INIT:
        with tpu_strategy.scope():
            model = RecommenderModel(n_users, n_animes, EMBEDDING_SIZES)
    else:
        model = RecommenderModel(n_users, n_animes, EMBEDDING_SIZES)

#     optimizer = Adam(lr=MAX_LR)
#     optimizer = Adam(lr=MAX_LR, decay=MAX_LR / EPOCHS)
    model.compile(loss='binary_crossentropy', metrics=['mae', 'mse'], optimizer='adam')
    
    return model

if not FUNCTIONAL_MODEL:
    model = get_model()

[INFO] Using Subclassing API Model


In [16]:
K = keras.backend
K.clear_session()

def RecommenderNet():
    # from Chaitanya's notebook https://www.kaggle.com/chaitanya99/recommendation-system-cf-anime/notebook
    embedding_size = 128
    
    user = Input(name = 'user', shape = [1])
    user_embedding = Embedding(name = 'user_embedding',
                       input_dim = n_users, 
                       output_dim = embedding_size)(user)
    
    anime = Input(name = 'anime', shape = [1])
    anime_embedding = Embedding(name = 'anime_embedding',
                       input_dim = n_animes, 
                       output_dim = embedding_size)(anime)
    
    # normalize=True will generate cosine similarity output
    # axes=2 to perform matrix multiplication on embedding axes
    x = Dot(name='dot_product', normalize=True, axes=2)([user_embedding, anime_embedding])
    x = Flatten()(x)
        
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("sigmoid")(x)
    
    model = Model(inputs=[user, anime], outputs=x)
    model.compile(loss='binary_crossentropy', metrics=["mae", "mse"], optimizer='Adam')
    
    return model

def get_functional_model():
    print("[INFO] Using Functional API Model")
    if TPU_INIT:    
        with tpu_strategy.scope():
            model = RecommenderNet()
    else:
        model = RecommenderNet()
    print(model.summary())
    return model
    
    
if FUNCTIONAL_MODEL:
    model = get_functional_model()

In [17]:
from tensorflow.keras.callbacks import Callback

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [18]:
WEIGHTS_PATH = 'weights.h5'
def load_trained_model():
    model = get_model()
    print('Calling model to load layers...')
    _ = model(tf.ones((1, 2)))
    model.load_weights(WEIGHTS_PATH)
    print('Loaded weights.')
    return model

In [19]:
%%time
model = load_trained_model()

[INFO] Using Subclassing API Model
Calling model to load layers...
Loaded weights.
CPU times: user 202 ms, sys: 136 ms, total: 338 ms
Wall time: 2.48 s


In [20]:
# load the best weights
try:
    if os.path.exists(checkpoint_filepath):
        model.load_weights(checkpoint_filepath)
except:
    print("[INFO] There is no checkpoint file")

[INFO] There is no checkpoint file


In [21]:
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    # because Dot layer was using normalize=True..?
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

anime_weights = extract_weights('anime_embedding', model)
user_weights = extract_weights('user_embedding', model)

In [22]:
anime_weights.shape, user_weights.shape

((17562, 128), (145311, 128))

In [23]:
anime_df = pd.read_csv('anime.csv')

In [24]:
anime_df.sort_values('Name', inplace=True)

In [25]:
all_anime_types = set(anime_df['Type'].unique())

def check_anime_types(types):
    types = set([types]) if isinstance(types, str) else set(types)
    if types.issubset(all_anime_types):
        return
    else:
        for anime_type in types:
            if anime_type not in all_anime_types:
                raise Exception(f'Anime type "{anime_type}" is not valid!')

def get_anime_rows(df, anime_query, exact_name=False, types=None):
    df = df.copy()
    if isinstance(anime_query, int):
        df = df[df.MAL_ID == anime_query]
    else:
        if exact_name:
            # get exact name
            df = df[df.Name == anime_query]
        else:
            df = df[df.Name.str.contains(anime_query, case=False, regex=False)]
        
    if types:
        check_anime_types(types)
        df = df[df.Type.isin(types)]
        
    return df

In [26]:
#pd.reset_option('all')
pd.set_option("max_colwidth", None)

def get_recommendation(anime_query, k=10, exact_name=False, types=None):
    if types:
        check_anime_types(types)
    anime_rows = get_anime_rows(anime_df, anime_query, 
                               exact_name=exact_name)
    if len(anime_rows) == 0:
        raise Exception(f'Anime not found for {anime_query}')
    anime_row = anime_rows.iloc[[0]]
    anime_id = anime_row.MAL_ID.values[0]
    anime_name = anime_row.Name.values[0]
    anime_idx = anime_id_to_idx.get(anime_id)

    weights = anime_weights
#         dists = np.dot(anime_weights[anime_idx], user_weights.T)
    distances = np.dot(weights, weights[anime_idx])

    sorted_dists_ind = np.argsort(distances)[::-1]

    print(f'Recommending animes for {anime_name}')
    display(anime_row.loc[:, 'MAL_ID': 'Aired'])

    anime_list = []
    # [1:] to skip the first row for anime_query
    for idx in sorted_dists_ind[1:]:
        similarity = distances[idx]
        anime_id = anime_idx_to_id.get(idx)
        anime_row = anime_df[anime_df.MAL_ID == anime_id]
        anime_type = anime_row.Type.values[0]
        if types and anime_type not in types:
            continue
        anime_name = anime_row.Name.values[0]
        score = anime_row.Score.values[0]
        genre = anime_row.Genres.values[0]

        anime_list.append({"Anime_id": anime_id, "Name": anime_name,
                           "Similarity": similarity, "Score": score,
                           "Type": anime_type, "Genre": genre
                          })
        if len(anime_list) == k:
            # enough number of recommendations
            break
    rec_df = pd.DataFrame(anime_list)
    return rec_df

In [27]:
get_recommendation('Dragon Ball Super', 10, types=('TV', 'Movie'))

Recommending animes for Dragon Ball Super


,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired
10623,30694,Dragon Ball Super,7.42,"Action, Adventure, Comedy, Super Power, Martial Arts, Fantasy, Shounen",Dragon Ball Super,ドラゴンボール超（スーパー）,TV,131,"Jul 5, 2015 to Mar 25, 2018"


,Anime_id,Name,Similarity,Score,Type,Genre
0,22777,Dragon Ball Kai (2014),0.699218,7.69,TV,"Action, Adventure, Comedy, Super Power, Martial Arts, Fantasy, Shounen"
1,31722,Nanatsu no Taizai: Seisen no Shirushi,0.650817,7.12,TV,"Action, Adventure, Ecchi, Fantasy, Magic, Shounen, Supernatural"
2,34577,Nanatsu no Taizai: Imashime no Fukkatsu,0.646382,7.84,TV,"Action, Adventure, Fantasy, Magic, Shounen, Supernatural"
3,24703,High School DxD BorN,0.609331,7.45,TV,"Action, Comedy, Demons, Ecchi, Harem, Romance, School"
4,34281,High School DxD Hero,0.603054,7.28,TV,"Action, Comedy, Demons, Ecchi, Harem, Romance, School"
5,22043,Fairy Tail (2014),0.595496,7.73,TV,"Action, Adventure, Comedy, Fantasy, Magic, Shounen"
6,19157,Youkai Watch,0.590198,6.54,TV,"Comedy, Demons, Kids, Supernatural"
7,21639,Yu☆Gi☆Oh! Arc-V,0.589715,6.79,TV,"Action, Game, Fantasy, Shounen"
8,35964,Basilisk: Ouka Ninpouchou,0.586572,5.47,TV,"Action, Drama, Historical, Martial Arts"
9,33834,Sin: Nanatsu no Taizai,0.586406,5.69,TV,"Ecchi, Fantasy"
